In [ ]:
# Makes it so any variable or statement on it's own line gets printed w/o print()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import scipy 

In [ ]:
pd.set_option('precision', 2)
pd.set_option('max_columns',10)

In [ ]:
# Import comma-seperated data from a text file
df = pd.read_csv('data\GSE88741-expression.csv', index_col=0)

# Bring in metadata from Excel
meta = pd.read_excel("data/GSE88741-metadata.xlsx", index_col=1)

In [ ]:
# Extract the Sample Titles and use them to replace the ugly GSM names
samples = meta.index
df.columns = samples

# Let's also make a new metadata column with cell line names
# Remove the last two characters from each sample name
samples = samples.str[:-2]
meta['cell_line'] = samples

In [ ]:
# Transpose the data so the observations (samples) are the rows and the variables (gene expression) are columns
df = df.transpose()
# Now merge the two tables with concat. 
melanoma_df = pd.concat([meta, df], axis=1)

# Functions to make DataFrames and ndarrays shine


Pandas and NumPy come with a [huge library of functions](http://pandas.pydata.org/pandas-docs/stable/reference/frame.html#) that work on ndarrays (nda) and DataFrames (df). Some of these are methods built into every nda and df object. Note that below I will refer to nda.method() or df.method(), which is just a way to say that those methods are built into every nda and df.  

We will cover a few of the most useful functions here and will see many more as the class goes on.

In this class we will introduce:
 - magic function %whos
 - computation methods min(), var(), mean(), etc.
 - sort_values()
 - groupby()
 - scipy.stats functions
     - pearsonr()   Pearson correlation coefficient
     - ttest_ind()  Student's t-test of independent variables
     - f_oneway()   ANOVA
 - And if we have time, np.random

### Magic functions (from base python)

Python has a set of "magic" commands that work on memory and the operating system. Not surprisingly, you can easily cause things to crash doing that, so magic commands limit your ability to do damage by only giving you a few powerful functions. All magic functions start with a "%" symbol. 

We will run into a few more of these later, but for now I just want to show you one really useful magic function: `%whos`.

You may find that you lose track of the variables you've created so far. Let's see whats there with `%whos`

In [ ]:
# This will show us every variable in memory
%whos

In [ ]:
# This will just show us every DataFrame
%whos DataFrame

In [ ]:
#If you're curious, here's how to list all available magic functions
%lsmagic

### NumPy and Pandas built in methods for ndarrays and DataFrames

There are hundreds of functions and built in methods for performing calculations with ndarrays and DataFrames, many that are rarely used (by me). Let's use a few of the most popular methods to standardize our data (we'll use this later for plotting) and find genes that are expressed at different levels in melanoma and normal cell lines.

We'll do this in four steps:
- Remove genes with low numbers of reads
- Standard scaling (subtract the mean, divide by the standard deviation)
- Sort on standard deviation to find the ten genes with the most (standardized) variation
- Apply statistical tests to get a p-value for normal vs melanoma 

### Remove columns with zero reads

In [ ]:
# Make a DataFrame of boolean values showing where there are zero reads
mel_zeros = None
mel_zeros

In [ ]:
# Add together all of the columns to get total number of samples with zero counts
zerosum = mel_zeros.None

In [ ]:
# We'll do more plotting later, but a simple histogram is useful here
zerosum.hist()

You can see that most genes either have reads for all 12 samples or none of them. We want to remove all of those genes from our DataFrame. Let's do that by selecting on a list of genes that have reads for all 12 samples.

In [ ]:
# Make a mask of all the columns we want to keep
# Let's be picky and only accept genes where all 12 samples have counts
keep = None

In [ ]:
# Using df.loc we can select just the genes we want to keep
keepers = melanoma_df.columns[None]
melanoma_low_removed = melanoma_df [None]
# Check the shape before and after
None
None

In [ ]:
melanoma_low_removed.head()

In [ ]:
# Let's strip out the metadata for now
melanoma_low_removed = melanoma_low_removed.loc[:,'A1BG':]

***
### <font color=brown>Hands on practice</font>
How many genes have less than 12 samples with reads, but more than zero?

In [ ]:
np.sum(zerosum <12) - np.sum(zerosum > 0)

***

### Standard scaling

Now that we've winnowed the table down to just the genes we're interested in, we will apply 'standard scaling' to our data, which removes the mean and scales to the standard deviation. 

$\ \frac{X-\bar{X}}{\sigma}$

where $\bar{X}$ is the mean and $\sigma$ is the standard deviation.

We can standardize our data in a few lines of code. 

In [ ]:
# Calculate the mean of our table with zero columns removed
mean = None
# and the standard deviation
stdev = None

In [ ]:
# Check out the head of each
None
None

In [ ]:
# To standardize our data we subtract the mean...
mel_standard = None
# And divide by the standard devaiation
mel_standard = None

In [ ]:
# Standardized data all have a mean of zero and a standard deviation of one
mel_standard.iloc[:,:5].std()
mel_standard.iloc[:,:5].mean()

OK, the data is standardized and ready to plot. We've used several new methods:
     - df.sum()   Sum
     - df.mean()  Mean
     - df.hist()  Histogram
     - df.std()   Standard deviation

Now let's find some genes with a lot of variance. This isn't the best way to find differentially expressed genes- we'll cover that in a few weeks.

### Finding high variance genes

In [ ]:
# Sort this Series and select the 10 genes with the largest variance
stdev.None
topvar = stdev[:9]
topvar

In [ ]:
# We want to use the gene names, not their variance, to filter the columns of melanoma_df
topvar.index

In [ ]:
# Now we can use that Series with melanoma_df.loc to make a subtable
topvartable = melanoma_df.loc[:,topvar.index]
topvartable

In [ ]:
# Pandas has a built in method for calculating basic statistics: df.describe()
None

### Grouping samples 

Our sample melanoma dataset has three replicates for each cell line. Those strains can be grouped together with 
```python 
df.groupby(by = ['column_name'])
```

However when we do that we get something a bit odd.

In [ ]:
mel_by_cel = melanoma_df.groupby('cell_line')
mel_by_cel

We have made a "DataFrameGroupBy" object. What is this?

This is an iterator, an object that iterates over a function, offering it one block of data at a time. To generate the mean of each gene for each cell line, we use the following:

In [ ]:
melanoma_df.groupby('cell_line').mean()

 We can similarly calculate the median, min, max, variance, etc. 
 
***
### <font color=brown>Hands on practice</font>
Calculate the mean and standard deviation of `topvartable` for each gene, grouped by cell line.

In [ ]:
# You'll need to merge the metadata and the topvartable
topvartable = None
# Then calculate the mean
mean_by_cell_line = None
variance_by_cell_line = None

In [ ]:
# Check the values
mean_by_cell_line
variance_by_cell_line

## Stats with scipy

The `scipy` library expands the `numpy` suite of mathematical functions. Like `numpy`, these are broken up into sublibraries. We'll use the stats sublibary to find the correlation between samples, run a t-test on our highly variable genes, and use ANOVA on one of those genes. 

In [ ]:
import scipy
from scipy import stats

Note that we used a new method of importing. Some libraries, like scipy, have multiple sub-libraries. You can import just the sub-library using the `from` command. This saves memory and time. 

We are mostly going to use the stats sublibrary, but we imported all of scipy so we can look over the help file and functions.

In [ ]:
# As we've done before, check out the new libraries

#scipy.
#scipy?
#stats.
#stats?

In [ ]:
# Pearson correlation coefficients can provide a quick estimate of the similarity between samples
# The function returns two values, so we provide two variable names to store the results
# The following line compares two normal cell lines. 
corr, p = stats.pearsonr(melanoma_low_removed.loc['FM_1',:], melanoma_low_removed.loc['FM_2',:])
corr
p

***
### <font color=brown>Hands on practice</font>
Find the correlation between FM_1 and UACC_62_1.

In [ ]:
# How good is the correlation between FM_1 and UACC_62_1?
None

***

In [ ]:
# Separate the TYRP1 expression values into normal vs metastatic
normal = topvartable.iloc[:3,0]
metastatic = topvartable.iloc[3:6,0]
normal
metastatic

In [ ]:
# let's test the significance of the difference in means with a t-test 
stat, p = stats.ttest_ind(normal, metastatic)
stat
p

In [ ]:
# ANOVA lets us see if any one of the cell lines has a significant difference in the mean
FM = topvartable.iloc[:3,0]
SK_MEL_147 = topvartable.iloc[3:6,0]
SK_MEL_28 = topvartable.iloc[6:9,0]
UACC_62 = topvartable.iloc[9:,0]

stats.f_oneway(FM, SK_MEL_147, SK_MEL_28, UACC_62)

***
### <font color=brown>Hands on practice</font>
Write a for loop to see how many of the topvartable genes have at least one cell line with significantly different means

In [ ]:
for i in np.arange(None):
    FM = None
    SK_MEL_147 = None
    SK_MEL_28 = None
    UACC_62 = None

    stat, p = None
    print(topvartable.columns[i],":",p)

### Random number generators

Random numbers are a good way of simulating expected results or sampling a random subset of data. 

NumPy stores these functions in the np.random sub-library. That makes it a little confusing as you'll have to repeat yourself a bit.

Random numbers can be taken from a uniform distribution (all numbers equally possible) or from a normal distribution (a 'bell-shape' centered on the mean) or many other distributions we won't cover here. 

For the rest of the class, try out these random number generators.

In [ ]:
# We will start by setting a random seed so that all our random variables match
np.random.seed(42)

In [ ]:
# A random integer with randint(start, stop(not included), number of values desired)
np.random.randint(1, 11, 9)

In [ ]:
# Random integers between 0 and 10 in a 2 by 2 array
print(np.random.randint(0, 10, size=[2,2]))

In [ ]:
# Three random floating-point number between 0 and 1
print(np.random.rand(3))

In [ ]:
# Normal distribution with mean=0 and variance=1 in a 1 by 5 array
print(np.random.randn(1, 5 ))

In [ ]:
# Pick 10 items from a given list, with equal probability
print(np.random.choice(['a', 'e', 'i', 'o', 'u'], size=10))  

# Pick 10 items from a given list with a predefined probability 'p'
print(np.random.choice(['a', 'e', 'i', 'o', 'u'], size=10, p=[0.3, .1, 0.1, 0.4, 0.1])) 